# Frequency tables
In this notebook we look at the wealth data from chapter 1, where the data are presented in summary form as a frequency table, with individuals amalgamated into classes of wealth.  Data in this form are a little more difficult to analyse using Python and there are some advantages to sticking to Excel for this.

However, we will show what can be done.  First we import the libraries we need, then the data.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
df = pd.read_csv('wealth.csv')

We check to see that that data have been correctly imported.  In this case we can look at all the data, as we have only have 13 classes.

In [5]:
df

,wealth_class,wealth,frequency,density
0,0-,5.0,1668,0.166800
1,"10,000-",17.5,1318,0.087900
2,"25,000-",32.5,1174,0.078300
3,"40,000-",45.0,662,0.066200
4,"50,000-",55.0,627,0.062700
5,"60,000-",70.0,1095,0.054800
6,"80,000-",90.0,1195,0.059800
7,"100,000-",125.0,3267,0.065300
8,"150,000-",175.0,2392,0.047800
9,"200,000-",250.0,2885,0.028900


The data look OK.  First we calculate the average wealth.  We cannot just calculate the average of the 'wealth' column as this will not take account of the different frequencies in each class.  To demonstrate:

In [6]:
df['wealth'].mean()

465.35714285714283

This is clearly not the right answer.  We could write some code to calculate the correct average, as one would do in Excel.  For example:

In [ ]:
df['tot_wealth'] = df['wealth'] * df['frequency']
ave_wealth = df['tot_wealth'].sum()/df['frequency'].sum()
print(ave_wealth)

The first line calculates a new column, containing the total wealth of each group.  The second line calculates the mean, effectively calculating $\frac{\Sigma fx}{\Sigma f}$.  

A simpler alternative is to use numpy's weighted average method, using the frequency column as weights. 

In [ ]:
np.average(df['wealth'], weights=df['frequency'])

This gives the same, correct answer.  Beyond this, things become more difficult.  There is not a simple command to calculate a standard deviation of grouped data, it has to be calculated as it would in Excel.  Here is some code that calculates the mean (again), variance and standard deviation.

In [ ]:
ave_wealth = df['tot_wealth'].sum()/df['frequency'].sum()
df['fx_squared'] = df['tot_wealth'] * df['wealth']                         # Create the 'fx_squared' column
var = (df['fx_squared'].sum()/df['frequency'].sum()) - ave_wealth**2       # Standard variance formula
sd = var**0.5
print('Mean =', ave_wealth, '\nVar =', var, '\nsd =', sd)

This gives us the correct answers but takes a little work (one could write a Python function to do all of it) and we still haven't got the median and quartiles.
We could, however, trick the computer by pretending we have all the individual observations.  For this we create 1668 rows of individuals with £5000 of wealth, 1318 observations of wealth £17500, etc.  It is not the same as the original survey data but then nor is the frequency table.
Doing this requires some code which we won't explain in detail.  (If interested, our source is here: https://serhiipuzyrov.com/2019/07/3-helpful-functions-for-data-manipulation-python/.

In [ ]:
def ungrouping(df, freq):
    df = df.copy(deep=True)
    df = df.loc[np.repeat(df.index.values,df[freq])
               ].reset_index(drop=True)
    df[freq] = 1
    return df

df2 = ungrouping(df, 'frequency')
df2

This has created a new dataframe, df2, which has 18677 observations, which is the same as the sum of the frequencies in the frequency table.  Now we can simply use the 'describe' method to get our summary statistics.

In [ ]:
pd.set_option('precision', 3)
df2['wealth'].describe()

This gives the correct answers for the mean and standard deviation.  The median and quartiles are approximately the same as in the book, the differences due to the fact that 'describe' is using the midpoint of the relevant class interval whereas our formula in the book allows us to obtain a better estimate within the interval.  The figures here are still useful as a fairly accurate guide.

## The bar chart
How we can obtain a bar chart or histogram of the data?  A bar chart, as in Figure 1. of the book, is easy to produce.

In [ ]:
plt.bar(df['wealth_class'], df['frequency'])

The labels on the x-axis need attention as they are currently unreadable but since we know this graph is inaccurate we will not bother to correct it here.
We could do slightly better by plotting the frequency densities rather than the frequencies.  Again, this is easy.

In [ ]:
plt.bar(df['wealth_class'], df['density'])

This gives a slightly better picture, at least it is less misleading.  Drawing a correct histogram would require stretching the x-axis, for which the code would be complicated.  Hence we do not go further into this.

## Suggested exercises
1. Write some code to calculate the skewness statistic, as in the book (equation 1.28).  You should get the same answer, 5.898.  Be careful to use dataframe df, not df2.  (Hint: calculate a new column 'fx_cubed' then adapt the code above for the variance and sd.
2. Add a title to the bar chart and try to fix the labels on the x-axis.